# Querying tables

> Objectives:
> * Compare queries of tabular data for **in-memory** containers
> * Compare sizes and times for those

In [ ]:
from ipython_memwatcher import MemWatcher
mw = MemWatcher()
mw.start_watching_memory()

In [ ]:
import os
dset = 'movielens-1m'
fdata = os.path.join(dset, 'ratings.dat.gz')
fitem = os.path.join(dset, 'movies.dat')

In [ ]:
import pandas as pd
# pass in column names for each CSV
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(fdata, sep=';', names=r_cols, compression='gzip')

m_cols = ['movie_id', 'title', 'genres']
movies = pd.read_csv(fitem, sep=';', names=m_cols,
                     dtype={'title': object, 'genres': object})

In [ ]:
movies.dtypes

In [ ]:
lens = pd.merge(movies, ratings)

In [ ]:
size_pandas = mw.measurements.memory_delta
size_pandas

In [ ]:
# pandas also comes with its own tool for getting memory usage
print(lens.memory_usage())

### Exercise

1) Why do you think that the size is the same for every column in the dataframe?

*Hint:* pandas stores the string columns in NumPy containers with 'object' dtype.

2) Provided how the `lens` DataFrame looks like:

In [ ]:
lens.head()

In [ ]:
lens.info()

Do you think this is all the memory that the DataFrame tables?

### Solution

In [ ]:
# Use deep=True for including the size of the object types
print(lens.memory_usage(deep=True))

In [ ]:
# For a more realistic memory usage, we are going to use deep=True
size_pandas2 = lens.memory_usage(deep=True).sum() / 2**20.
size_pandas2

## Querying a pandas DataFrame

In [ ]:
# Let's do a typical query in pandas and see how fast it can be
t = %timeit -o lens.query("(title == 'Tom and Huck (1995)') & (rating == 5)")['user_id']
result = lens.query("(title == 'Tom and Huck (1995)') & (rating == 5)")['user_id']
result

In [ ]:
# Store it for future reference
qtime_pandas = t.best

## Use a compressed in-memory container via `bcolz`

In [ ]:
import bcolz
bcolz.print_versions()
bcolz.defaults.cparams['cname'] = 'lz4'
bcolz.defaults.cparams['clevel'] = 9
bcolz.defaults.cparams['shuffle'] = bcolz.BITSHUFFLE  # try with NOSHUFFLE and BITSHUFFLE as well
bcolz.set_nthreads(1)

In [ ]:
zlens = bcolz.ctable.fromdataframe(lens)

In [ ]:
zlens

In [ ]:
size_bcolz = mw.measurements.memory_delta
size_bcolz

We see that the reported memory consumption is *really* low, so let's take the internal estimation for comparison purposes.

In [ ]:
zlens

In [ ]:
size_bcolz2 = zlens.cbytes / 2**20.

In [ ]:
size_pandas2 / size_bcolz2

## Querying a bcolz ctable

In [ ]:
# Workaround to use bytes instead of Unicode (not supported yet in numexpr queries)
title = zlens['title'][:].astype('S82')
zlens.delcol('title')
zlens.addcol(title, 'title')

In [ ]:
# Workaround to use bytes instead of Unicode (not supported yet in numexpr queries)
genre = zlens['genres'][:].astype('S47')
zlens.delcol('genres')
zlens.addcol(genre, 'genres')

We can see that the space taken by a bcolz container is around 20x smaller (!) than a pandas one.

In [ ]:
# Querying a bcolz dataset
resultz = [(r.nrow__, r.user_id) for r in zlens.where("(title == b'Tom and Huck (1995)') & (rating == 5)", outcols=['nrow__', 'user_id'])]
t = %timeit -o [(r.nrow__, r.user_id) for r in zlens.where("(title == b'Tom and Huck (1995)') & (rating == 5)", outcols=['nrow__', 'user_id'])]

In [ ]:
qtime_bcolz = t.best

In [ ]:
qtime_pandas / qtime_bcolz

We see that by using a compressed bcolz container, it provides similar query times than pandas.

In [ ]:
# Sanity check (always check for your results!)
print("results with pandas Dataframe:", result)
print("results with bcolz ctable:", resultz)

## Use a *uncompressed* in-memory container via `bcolz`

So, what happens when we use an uncompressed container in bcolz?  We do expect better speed?  If so, how much?  Let's measure:

In [ ]:
bcolz.defaults.cparams['clevel'] = 0

In [ ]:
uzlens = bcolz.ctable.fromdataframe(lens)
uzlens

In [ ]:
size_ubcolz = mw.measurements.memory_delta
size_ubcolz

In [ ]:
size_ubcolz2 = uzlens.cbytes / 2**20.
size_ubcolz2

In [ ]:
size_pandas2 / size_ubcolz2

We can see that the space taken by an uncompressed bcolz container is very close to a pandas one (but keep an eye on the reported memory usage by memory_watcher).

In [ ]:
# Workaround to use bytes instead of Unicode (not supported yet in numexpr queries)
title = zlens['title'][:].astype('S82')
uzlens.delcol('title')
uzlens.addcol(title, 'title')

In [ ]:
# Querying a bcolz dataset
resultz = [(r.nrow__, r.user_id) for r in uzlens.where("(title == b'Tom and Huck (1995)') & (rating == 5)", outcols=['nrow__', 'user_id'])]
t = %timeit -o [(r.nrow__, r.user_id) for r in uzlens.where("(title == b'Tom and Huck (1995)') & (rating == 5)", outcols=['nrow__', 'user_id'])]

In [ ]:
qtime_ubcolz = t.best

In [ ]:
qtime_pandas / qtime_ubcolz

## Using structured NumPy arrays

In [ ]:
nalens = zlens[:]

In [ ]:
size_numpy = mw.measurements.memory_delta
size_numpy

In [ ]:
nalens

In [ ]:
size_numpy2 = round((nalens.size * nalens.dtype.itemsize) / 2**20., 3)
size_numpy2

In [ ]:
# But, again, this does not account for 'O'bject dtypes, which are stored in its own storage area
# So, let's take this into account
import sys
size_numpy3 = size_numpy2 + round(sum(sum(sys.getsizeof(o) for o in nalens[col]) for col in ('title', 'genres')) // 2**20., 3)
size_numpy3

In [ ]:
resultna = nalens[(nalens['title'] == 'Tom and Huck (1995)') & (nalens['rating'] == 5)]
t = %timeit -o nalens[(nalens['title'] == 'Tom and Huck (1995)') & (nalens['rating'] == 5)]
resultna

Again, NumPy works the fastest for in-memory data containers, while memory consumption is close to pandas (a hair less actually).

In [ ]:
qtime_numpy = t.best

## Performance comparsion

In [ ]:
%matplotlib inline

In [ ]:
# Query times
labels = ["pandas", "bcolz", "uncompressed bcolz", "numpy (recarray)"]
df = pd.DataFrame({'time (sec)': [qtime_pandas, qtime_bcolz, qtime_ubcolz, qtime_numpy]}, index=labels)
pd.options.display.mpl_style = 'default'
df.plot(kind='barh', figsize=(12,5), fontsize=16, title="Query times for MovieLens 1m (in-memory)")

## Size comparison

In [ ]:
# Container sizes
df = pd.DataFrame({'size (MB)': [size_pandas2, size_bcolz2, size_ubcolz2, size_numpy3]}, index=labels)
pd.options.display.mpl_style = 'default'
df.plot(kind='barh', figsize=(12,5), fontsize=16, title="Container sizes for MovieLens 1m")

## Rules of thumb for querying in-memory tabular datasets

* Choose pure NumPy recarrays if you need the fastest speed
* Choose bcolz ctables if you need to store lots of data in limited memory and not want to loose too much speed
* Choose pandas if what you need is rich functionality on top of your containers (at the penalty of some speed)